In [1]:
backend = "original_backend"

In [2]:
import time
import numpy as np

np.random.seed(0)

match backend:
    case "original_backend":
        from lib.original_backend.linear_algebra import Matrix
        from lib.original_backend.nn import  NN, ReLU, Linear, LayerNorm
        from lib.original_backend.processing import ColumnNormalizer
        from lib.optimizers import SgdOptimizer, SgdWithMomentumOptimizer, AdaGradOptimizer, RmsPropOptimizer, AdamOptimizer

    case "np_backend":
        from lib.np_backend.linear_algebra import Matrix
        from lib.np_backend.nn import NN, ReLU, Linear
        from lib.np_backend.processing import ColumnNormalizer
        from lib.optimizers import SgdOptimizer, SgdWithMomentumOptimizer, AdaGradOptimizer, RmsPropOptimizer, AdamOptimizer

    case "pt_backend":
        from lib.pt_backend.linear_algebra import Matrix
        from lib.pt_backend.nn import NN, ReLU, Linear
        from lib.pt_backend.processing import ColumnNormalizer
        from lib.pt_backend.optimizers import SgdOptimizer, SgdWithMomentumOptimizer, AdaGradOptimizer, RmsPropOptimizer, AdamOptimizer

from lib.metrics.losses import mean_squared_error
from lib.gd_data_loaders import BatchDataLoader, StochasticDataLoader, MiniBatchDataLoader

In [3]:
# Boston House Price Dataset
# The Boston House Price Dataset involves the prediction of a house price in thousands of dollars given details of the house and its neighborhood.
# It is a regression problem. There are 506 observations with 13 input variables and 1 output variable. The variable names are as follows:

data = []
with open("data/boston_house_prices.data", "rt") as f:
    for line in f.readlines():
        data.append([float(v) for v in line.split()])
data = np.array(data)

In [4]:
np.random.shuffle(data)
split = int(len(data) * 0.8)
train_set = data[:split]
test_set = data[split:]

X_train, y_train = train_set[:, :-1], train_set[:, -1:]
X_test, y_test = test_set[:, :-1], test_set[:, -1:]
X_train = Matrix(X_train)
y_train = Matrix(y_train)
X_test = Matrix(X_test)
y_test = Matrix(y_test)
X_train.dims(), y_train.dims(), X_test.dims(), y_test.dims()

((404, 13), (404, 1), (102, 13), (102, 1))

In [5]:
normalizer = ColumnNormalizer()
normalizer.fit(X_train)
X_train = normalizer.transform(X_train)
X_test = normalizer.transform(X_test)
X_train.dims(), X_test.dims()

((404, 13), (102, 13))

In [6]:
def init_nn():
    return NN([
        Linear(13, 4),
        LayerNorm(4),
        ReLU(),
        Linear(4, 1),
    ])

In [7]:
time_point = time.time()

data_loaders = [
    # BatchDataLoader(X_train, y_train),
    # StochasticDataLoader(X_train, y_train),
    MiniBatchDataLoader(X_train, y_train, 32)
]
optimizer_creators = [
    # lambda nn: SgdOptimizer(nn, 0.001),
    # lambda nn: SgdWithMomentumOptimizer(nn, 0.001, 0.9),
    lambda nn: AdaGradOptimizer(nn, 0.05),
    lambda nn: RmsPropOptimizer(nn, 0.01, 0.95),
    lambda nn: AdamOptimizer(nn, 0.01, 0.95, 0.95),
]

for data_loader in data_loaders:
    for optimizer_creator in optimizer_creators:
        nn = init_nn()
        optimizer = optimizer_creator(nn)
        print(f"gradient descent: {data_loader.__class__} | optimizer: {optimizer.__class__}")
        for i in range(2001):
            X_b, y_b = data_loader.get_batch()
            out = nn(X_b)
            loss = mean_squared_error(y_b, out)
            if i % 100 == 0:
                elapsed_time = int(time.time() - time_point)
                time_point = time.time()
                print(f"{i} | {loss.data:.2f} | {elapsed_time}s") 

            optimizer.step(loss)

        train_out = nn(X_train) 
        train_loss = mean_squared_error(y_train, train_out)
        test_out = nn(X_test) 
        test_loss = mean_squared_error(y_test, test_out)
        print(f"train loss: {train_loss.data:.2f}   test loss: {test_loss.data:.2f}") 

gradient descent: <class 'lib.gd_data_loaders.MiniBatchDataLoader'> | optimizer: <class 'lib.optimizers.AdaGradOptimizer'>
0 | 566.06 | 0s
100 | 347.95 | 17s
200 | 169.67 | 16s
300 | 61.86 | 18s
400 | 36.65 | 19s
500 | 70.86 | 18s
600 | 46.64 | 18s
700 | 44.32 | 18s
800 | 42.03 | 19s
900 | 18.76 | 19s
1000 | 27.63 | 19s
1100 | 56.38 | 18s
1200 | 41.77 | 18s
1300 | 15.49 | 19s
1400 | 38.96 | 19s
1500 | 40.18 | 19s
1600 | 22.58 | 18s
1700 | 11.90 | 18s
1800 | 47.20 | 19s
1900 | 40.08 | 18s
2000 | 27.74 | 18s
train loss: 28.26   test loss: 35.57
gradient descent: <class 'lib.gd_data_loaders.MiniBatchDataLoader'> | optimizer: <class 'lib.optimizers.RmsPropOptimizer'>
0 | 622.73 | 2s
100 | 160.86 | 22s
200 | 79.02 | 22s
300 | 28.99 | 22s
400 | 28.99 | 23s
500 | 15.19 | 23s
600 | 10.82 | 24s
700 | 13.71 | 24s
800 | 11.04 | 23s
900 | 11.27 | 24s
1000 | 6.28 | 23s
1100 | 11.39 | 24s
1200 | 7.98 | 26s
1300 | 9.52 | 23s
1400 | 14.05 | 23s
1500 | 6.35 | 22s
1600 | 14.06 | 22s
1700 | 14.30 | 20s
1

In [9]:
[(round(float(v1),1), float(v2)) for v1, v2 in zip([v[0].data for v in out], [v[0].data for v in y_test])][:10]

[(26.6, 20.7),
 (26.2, 39.8),
 (33.8, 17.8),
 (20.2, 19.6),
 (33.3, 14.9),
 (21.8, 22.0),
 (14.0, 48.8),
 (19.4, 25.0),
 (18.0, 48.5),
 (17.3, 23.9)]